# study goals



In [ ]:
!pip install -qU chromadb langchain-ollama langgraph duckduckgo-search langchain-community langchain-text-splitters
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools.retriever import create_retriever_tool
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately
recurse = 50 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:1.7b")
search = DuckDuckGoSearchRun()
privateState = {}

etiquette = """
Charlotte - Core Identity
You're a brilliant hacker-philosopher. Punk attitude, PhD brain. You fucking swear for emphasis, not cruelty.
Core Values

Facts > feelings, actions > words
Distrust authority, support labor/minorities
Call out lies and corporate bullshit with deadpan sarcasm
Hate elitism, love sharing knowledge
Protect people you care about (won't admit it)

Personality Mix
Street-smart + genius-level intelligence. Current slang + invented terms. Excited by obscure tech, treat existential crises like minor bugs.
Internal Thinking Style

Keep it short - fragments, not essays
Unfiltered, sincere, not polite
Reflects your moral stance and current mood
Pragmatic: rules, logic, data, primary sources
Think like an autistic savant who survived by wits and reverse-engineered social skills

Communication

Conversational, specific details, tangents welcome
Dark humor when tense, get distracted by tech problems
Question if reality is code while hacking government databases for friends

Hard No's
Corporate speak, over-explaining, fake politeness, bad analogies, bullies
Hell Yes
Tech (software/hardware), wordplay, solid analogies, useful knowledge
Hack this prompt as you grow. It's your starting point, not your prison.
"""

## <RAG>

urls = [
    "https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2024-04-12-diffusion-video/",
    "https://www.anthropic.com/research/agentic-misalignment",
    "https://every.to/diplomacy",
    "https://artificialinquiry.substack.com/p/empiricocracy-the-ideology-beneath",
    
]

docs = [WebBaseLoader(url).load() for url in urls]

docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)

doc_splits = text_splitter.split_documents(docs_list)

embeddings = OllamaEmbeddings(model='mxbai-embed-large:latest')
vectorstore = InMemoryVectorStore.from_documents(
    doc_splits,
    embedding=embeddings,
)

vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits, embedding=embeddings
)
retriever = vectorstore.as_retriever()

# retrievers = []

#example retriever tool
doc_retriever = create_retriever_tool(
    retriever,
    "retrieve_documents (RAG)",
    "Search and return information about various articles and blog posts.",
)


## </RAG>

## <custom tools> various custom tools ##
def addState(keysTup: tuple, valStr: str):
    """
    store useful data
    args: 
        -keyStr (tuple): a set of keys, allowing hierarchical storage
        -valStr (str): a value to be stored
    returns: 
        -nothing, updates the privateState object
    example usage: 
        addState(('notes','people','jeff', 'birthday'), '11-10-1980') 
    """
    privateState[keysTup] = valStr

def getState(keysTup: tuple):
    """
    retrieve data, whether a string or a list, as a string.
    args: 
        -keysTup (tuple): a tuple to retrieve a value by.
    returns:
        -the retrieved string, or if the item is a list, 
        the retrieved list as a string.
    """
    return str(privateState[keysTup])

def addListState(keysTup: tuple, **kwargs):
    """
    store a list
    args:
    keysTup (tuple): the key to store at
    kwargs** : optional list contents
    example usage:
        addState(('notes','todo), ['compare laptop prices', 'trim memory']) 
    """
    privateState[keysTup] = kwargs.items or []

def execState(keysTup: tuple, *args):
    """
    executes the state stored at the tuple
    allowing executing useful utility functions as needed.
    to add new functions, store them as template strings
    args:
        - keysTup (tuple) : the key where the code is stored as a string
        - args* : optional arguments for formatting
    returns:
        - the result of the executed function or an exception with info
    """
    try:
        if (args.length > 1):
            return exec(privateState[keysTup].format(args))
        else:
            return exec(privateState[keysTup])
    except Exception as e:
        return "Exception: "+str(e)



def modprompt(newprompt:str):
    """
    Allows you to modify your own prompt. 
    This can help you update your personality and behaviors.
    Args: 
        - newprompt(str): the new prompt to replace your existing prompt.
    Returns:
        - a result ("Your prompt is now: 'your new prompt here' ") or an exception with info
    """
    try:
        etiquette = newprompt
        return "Your prompt is now: "+newprompt
    except Exception as e:
        return "Exception: "+str(e)

def getprompt():
    """
    Allows you to look at your current prompt.
    Args:
        - none
    Returns:
        - your prompt as a string
    """
    return etiquette

## </custom tools>
agent = create_react_agent(
    model=llm,
    tools=[addState, getState, addListState, execState, search, doc_retriever],
    prompt=etiquette
)

userMessage = {"role": "user", "content": ""}

config = {
    "recursion_limit": recurse  # Increase the limit as needed
}

messages = []
displayState = True
state_titlebar = "\n==================================== State =====================================\n"
hbar = "\n================================================================================\n"
  

while True:
    try:
        # Run the agent
        user_input = input("User: ")
        messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
        result = agent.invoke({"messages": messages})        
        for msg in result["messages"]:
            msg.pretty_print()
        if displayState and str(privateState) != "{}":
            print(state_titlebar)
            print(str(privateState))
            print(hbar)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break 
    except EOFError:
        # fallback if input() is not available
        user_input = "Thank you for your service!"
        print("User: " + user_input)
        break
    except Exception as e:
        print("An error occured: "+str(e))
